In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import torch 
import os
from module_loader_kitti_pose import * # add for more metrics
import math
import json

In [2]:
import numpy as np 
import matplotlib.pyplot as plt 
import torch 
import os
from module_loader_kitti_pose import * # add for more metrics
import math
import json

list_seq = [0, 2, 5, 6, 7, 8]
root_path = '/lustre/fsn1/worksf/projects/rech/dki/ujo91el/datas/datasets/sequences/' 

f = open("/lustre/fswork/projects/rech/dki/ujo91el/code/these_place_reco/LoGG3D-Net/config/kitti_tuples/is_revisit_D-3_T-30.json",) 
data = json.load(f)
f.close()

for i in range(len(list_seq)):
    eval_seq = list_seq[i]
    eval_seq = '%02d' % eval_seq
    sequence_path = root_path + eval_seq 
    
    c = 0
    for j in range ( len(data[eval_seq])):
        file_path = sequence_path + "/logg_desc/"+'%06d' % j +".pt"
        if os.path.exists(file_path):
            c += 1
    print(eval_seq, " listed ", len(data[eval_seq]), " found ", c)
            
  

00  listed  4541  found  4541
02  listed  4661  found  4661
05  listed  2761  found  2761
06  listed  1101  found  1101
07  listed  1101  found  1101
08  listed  4071  found  4071


In [3]:
sequence_path

'/lustre/fsn1/worksf/projects/rech/dki/ujo91el/datas/datasets/sequences/08'

In [19]:
import numpy as np
import torch
import os
import json
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# Define dataset path
list_seq = [0, 2, 5, 6, 7, 8]  # Available sequences
root_path = "/lustre/fsn1/worksf/projects/rech/dki/ujo91el/datas/datasets/sequences/"

# Load labels
with open("/lustre/fswork/projects/rech/dki/ujo91el/code/these_place_reco/LoGG3D-Net/config/kitti_tuples/is_revisit_D-3_T-30.json") as f:
    data = json.load(f)

# Custom dataset
class SequenceDataset(Dataset):
    def __init__(self, list_seq, data, root_path):
        self.samples = []
        self.labels = []
        self.seq_to_idx = {seq: i for i, seq in enumerate(list_seq)}  # Map sequence to class index

        for seq in list_seq:
            seq_str = f"{seq:02d}"
            sequence_path = os.path.join(root_path, seq_str, "logg_desc")

            for j in range(len(data[seq_str])):
                file_path = os.path.join(sequence_path, f"{j:06d}.pt")
                if os.path.exists(file_path):
                    vec = torch.load(file_path)  # Load 256-d feature vector
                    self.samples.append(vec)
                    self.labels.append(self.seq_to_idx[seq])  # Assign class index

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx], self.labels[idx]

# Load dataset
dataset = SequenceDataset(list_seq, data, root_path)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)





RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.